In [1]:
import numpy as np, os, h5py, pandas as pd
np.set_printoptions(suppress=True)
import tensorflow as tf
from tensorflow import keras as tfk
from tfomics import tfomics 
from tfomics.tfomics import moana, impress, explain
import logomaker
from logomaker import Logo

from paramiko import SSHClient
from scp import SCPClient

from pairwise_conv_1d import PairwiseConv1D, PairwiseFromStdConv1D
from nearest_neighbor_conv_1d import NearestNeighborConv1D, NearestNeighborFromStdConv1D
from ho_regularizer import HigherOrderKernelRegularizer

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
%load_ext autoreload
%autoreload 2
tfk.backend.clear_session()

## Get data 

In [3]:
reverse_compliment = True
data_path = os.path.join('./../../../genomics_datasets/exponential_activations')
filepath = os.path.join(data_path, 'IRF1_400_h3k27ac.h5')
with h5py.File(filepath, 'r') as dataset:
    x_train = np.array(dataset['X_train']).astype(np.float32)
    y_train = np.array(dataset['Y_train']).astype(np.float32)
    x_valid = np.array(dataset['X_valid']).astype(np.float32)
    y_valid = np.array(dataset['Y_valid']).astype(np.int32)
    x_test = np.array(dataset['X_test']).astype(np.float32)
    y_test = np.array(dataset['Y_test']).astype(np.int32)

x_train = np.squeeze(x_train).transpose([0,2,1])
x_valid = np.squeeze(x_valid).transpose([0,2,1])
x_test = np.squeeze(x_test).transpose([0,2,1])
n = x_train.shape[0]

if reverse_compliment:
    x_train_rc = x_train[:,::-1,:][:,:,::-1]
    x_valid_rc = x_valid[:,::-1,:][:,:,::-1]
    x_test_rc = x_test[:,::-1,:][:,:,::-1]

    x_train = np.vstack([x_train, x_train_rc])
    x_valid = np.vstack([x_valid, x_valid_rc])
    x_test = np.vstack([x_test, x_test_rc])

    y_train = np.vstack([y_train, y_train])
    y_valid = np.vstack([y_valid, y_valid])
    y_test = np.vstack([y_test, y_test])

# create tensorflow dataset
trainset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
sampleset = tf.data.Dataset.from_tensor_slices((x_valid[:2], y_valid[:2]))

N, L, A = x_test.shape
num_labels = y_test.shape[1]

In [4]:
f get_model(input_shape, first_layer_conv, name='model', first_layer_activation='relu'):
    x = tfk.layers.Input(shape=input_shape, name='input')
    y = first_layer_conv(x)
    if first_layer_conv.use_bias:
        y = tfk.layers.BatchNormalization()(y)
    y = tfk.layers.Activation(first_layer_activation, name=first_layer_activation)(y)
    y = tfk.layers.MaxPool1D(pool_size=25)(y)
    y = tfk.layers.Dropout(0.1)(y)
    
    # layer 2 - convolution
    l2 = tfk.regularizers.l2(1e-4)
    y = tfk.layers.Conv1D(filters=48,kernel_size=7,strides=1,padding='same',kernel_regularizer=l2, )(y)        
    y = tfk.layers.Activation('relu')(y)
    y = tfk.layers.MaxPool1D(pool_size=4)(y)
    y = tfk.layers.Dropout(0.1)(y)

    # layer 3 - Fully-connected 
    y = tfk.layers.Flatten()(y)
    y = tfk.layers.Dense(96,use_bias=False,kernel_regularizer=l2)(y)      
    y = tfk.layers.BatchNormalization()(y)
    y = tfk.layers.Activation('relu')(y)
    y = tfk.layers.Dropout(0.5)(y)

    # Output layer
    y = tfk.layers.Dense(1, kernel_initializer='glorot_normal',name='logits')(y)
    y = tfk.layers.Activation('sigmoid')(y)

    # create keras model
    model = tfk.Model(inputs=x, outputs=y, name=name)
    
    return model

SyntaxError: invalid syntax (<ipython-input-4-9e459e4fda95>, line 1)